In [41]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model

In [42]:
def extract_mfcc(file_path):
    # Дууны файлыг унших
    y, sr = librosa.load(file_path, sr=None)
    
    # MFCC-г гаргаж авах
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # 13 MFCC шинж авах
    mfcc_mean = np.mean(mfcc.T, axis=0)  # Дундажлаад, онцлог болгон хадгалах
    
    return mfcc_mean

# Жишээ файлын зам
file_path = 'C:/Users/Acer/OneDrive/Desktop/UrbanSound8k/dx_cup/DXCUP/data/Ondorhaan 2.wav'
mfcc_features = extract_mfcc(file_path)

print("MFCC Features:", mfcc_features)

MFCC Features: [-335.88632     91.498024   -24.456554    15.036045     6.246258
    3.868974    -2.1248338  -10.607402   -10.757667    -1.9641104
   -8.978896     8.84592      0.787184 ]


In [43]:
def extract_spectral_contrast(file_path):
    y, sr = librosa.load(file_path, sr=None)
    
    # Spectral Contrast онцлог гаргаж авах
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    spectral_contrast_mean = np.mean(spectral_contrast.T, axis=0)  # Дундажлаад хадгалах
    
    return spectral_contrast_mean

spectral_contrast_features = extract_spectral_contrast(file_path)
print("Spectral Contrast Features:", spectral_contrast_features)


Spectral Contrast Features: [15.85487965  9.82125981 13.14154069 14.3605117  16.8704314  20.25621103
 37.03299586]


In [44]:
def extract_chroma(file_path):
    y, sr = librosa.load(file_path, sr=None)
    
    # Chroma онцлог гаргаж авах
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = np.mean(chroma.T, axis=0)  # Дундажлаад хадгалах
    
    return chroma_mean

chroma_features = extract_chroma(file_path)
print("Chroma Features:", chroma_features)


Chroma Features: [0.41869605 0.4472138  0.4450555  0.42712587 0.42234325 0.47550958
 0.49347407 0.5271733  0.546242   0.51172346 0.43266034 0.39682218]


In [45]:
def extract_features(file_path):
    mfcc_features = extract_mfcc(file_path)
    chroma_features = extract_chroma(file_path)
    spectral_contrast_features = extract_spectral_contrast(file_path)
    
    # Бүгдийг нэгтгэх
    features = np.hstack((mfcc_features, chroma_features, spectral_contrast_features))
    return features

# Онцлогийг гаргаж авах
all_features = extract_features(file_path)
print("Combined Features:", all_features)


Combined Features: [-335.88632202   91.49802399  -24.45655441   15.03604507    6.24625778
    3.86897397   -2.12483382  -10.60740185  -10.75766659   -1.96411037
   -8.97889614    8.84591961    0.787184      0.41869605    0.4472138
    0.44505551    0.42712587    0.42234325    0.47550958    0.49347407
    0.52717328    0.546242      0.51172346    0.43266034    0.39682218
   15.85487965    9.82125981   13.14154069   14.3605117    16.8704314
   20.25621103   37.03299586]


In [ ]:
def extract_features(file_path):
    # MFCC
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc.T, axis=0)

    # Chroma
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = np.mean(chroma.T, axis=0)

    # Spectral Contrast
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    spectral_contrast_mean = np.mean(spectral_contrast.T, axis=0)

    # Бүгдийг нэгтгэх
    features = np.hstack((mfcc_mean, chroma_mean, spectral_contrast_mean))
    return features

# Дууны өгөгдлийн директори
data_dir = 'DXCUP/data'
data = []  # Дата массив

# Дууны өгөгдлийг боловсруулж, онцлог гаргаж авах
for filename in os.listdir(data_dir):
    if filename.endswith('.wav'):
        audio_path = os.path.join(data_dir, filename)
        features = extract_features(audio_path)

        # Файлын нэрийн сүүлчийн тоог шалгах
        file_number = filename.split('.')[0]  # .wav-г хасаж, файлын нэрийг авна
        if file_number[-1].isdigit() and int(file_number[-1]) % 2 == 0:  # Тэгш тоо шалгах
            label ='cold' # "ханиад"
        else:
            label = 'breath'  # амьсгал

        # Файлын нэр, шошго, онцлогийг хадгалах
        data.append([filename, label] + features.tolist())

# CSV файлд хадгалах
columns = ['filename', 'label'] + [f'mfcc_{i+1}' for i in range(13)] + \
          [f'chroma_{i+1}' for i in range(12)] + \
          [f'spectral_contrast_{i+1}' for i in range(7)]  # 7 нь spectral contrast онцлогийн тоо

df = pd.DataFrame(data, columns=columns)
df.to_csv('Sound.csv', index=False)  # CSV файл руу хадгалах

print("Онцлог шинж чанарууд болон шошго CSV файлд хадгалагдлаа.")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dx_cup/DXCUP/data'

In [47]:
df = pd.read_csv('Sound.csv')

# Онцлог шинж чанар (features) болон шошго (labels) ялгах
X = df.drop(columns=['filename', 'label'])  # Онцлог шинж чанарууд
y = df['label']  # Шошго

# Сургалт, тестийн өгөгдөл хуваах
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Логистик Регрессийн загвар үүсгэх
model = LogisticRegression(max_iter=1000)  # max_iter нь алгоритмийг хэт удаан ажиллуулахгүй байх

# Загварыг сургалтанд оруулах
model.fit(X_train, y_train)

# Загвараа шалгах
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Загварын үнэн зөвийн түвшин: {accuracy * 100:.2f}%")

Загварын үнэн зөвийн түвшин: 100.00%
